# 网络结构

YOLOS
![image-20230407上午101700351](../配图/Transformer家族.assets/image-20230407上午101700351.png)




ViT
<img src="../配图/Transformer家族.assets/image-20220812下午35357981.png" alt="image-20220812下午35357981" style="zoom: 67%;" />

yolos 的结构是在 ViT 的基础上做了改变，从图中可以看出主干网络结构是和 vit 一致的，只不过 yolos 作为检测任务，他在 vit 的基础上concat了 100 个 det token，

所以进入 encode 的输入是:
$$
z_0 = [x_{PATCH}^1 E,x_{PATCH}^2 E,...x_{PATCH}^N E,x_{DET}^{100}]+P
$$
其中 $x_{PATCH}^i E$ 就是普通 vit 的输入，$x_{DET}^{j}$ 就是100 个可以学习的 det，$P\in (N+100,D)$ 是增加的位置编码，来保留输入的位置信息。



模型的特征的输出依旧是$[1,N+100,D]$的尺寸

```Python
return x[:, -self.det_token_num:, :]
```





det 是一个 1,100,768 的尺寸，它是随机初始化的，以避免2D结构的归纳偏置和标签分配过程中注入任务的先验知识。

```Python
self.det_token = nn.Parameter(torch.zeros(1, det_token_num, self.embed_dim))   # 1,100,768
self.det_token = trunc_normal_(self.det_token, std=.02)
```





将这个特征输出进入两个参数不共享的 MLP。

```Python
self.class_embed = MLP(hidden_dim,hidden_dim,output_dim=num_classes+ 1,num_layers=3)

self.bbox_embed = MLP(hidden_dim,hidden_dim,output_dim=4,num_layers=3)
```







MLP包含两个隐藏层，中间带有ReLU，每层都不会改变尺寸。

```Python
class MLP(nn.Module):
    """ Very simple multi-layer perceptron (also called FFN)"""

    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super().__init__()
        self.num_layers = num_layers
        h = [hidden_dim] * (num_layers - 1)
        self.layers = nn.ModuleList(nn.Linear(n, k) for n, k in zip([input_dim] + h, h + [output_dim]))

    def forward(self, x):
        for i, layer in enumerate(self.layers):
            # 除了最后一层，每层都有 ReLU
            x = F.relu(layer(x)) if i < self.num_layers - 1 else layer(x)
        return x
```





检测的输出就是 MLP 输出结果的 sigmoid，模型最终就输出类别和检测

```Python
outputs_class = self.class_embed(x)
outputs_coord = self.bbox_embed(x).sigmoid()
out = {'pred_logits': outputs_class, 'pred_boxes': outputs_coord}
        return out
```





## 微调操作

所有参数都从ImageNet-1k预训练的权重初始化，**除了两个MLP头以及100个随机初始化的det token。**



模型使用了 imagenet 进行了预训练，随后使用了 coco 数据集进行微调，但是coco 数据集的分辨率比 imagenet 更大，所以还是保持了 16*16 个 patch不变，但是这样的尺寸就会变大，**所以使用 2d 插值来讲位置编码变得和输入尺寸一样大。**



这样 N 变大了，但是768 的尺寸不变，依旧可以将 patch token，det token，cls 拼接在一起。

```Python
patch_pos_embed # 1,196,768
img_size = [800,1344]

patch_pos_embed = patch_pos_embed.transpose(1,2)  # 1,768,196
B, E, Q = patch_pos_embed.shape # 1 768 196
P_H, P_W = self.img_size[0] // self.patch_size, self.img_size[1] // self.patch_size  # 14,14

patch_pos_embed = patch_pos_embed.view(B, E, P_H, P_W) # 1,768,14,14
H, W = img_size
new_P_H, new_P_W = H//self.patch_size, W//self.patch_size # 50,84

# 双线性插值
patch_pos_embed = nn.functional.interpolate(patch_pos_embed, size=(new_P_H,new_P_W), mode='bicubic', align_corners=False) # 1,768,50,48


patch_pos_embed = patch_pos_embed.flatten(2).transpose(1, 2) # 1,4200,768

```

